In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/'CS685'/Project/
!ls

Mounted at /gdrive
/gdrive/MyDrive/CS685/Project
'Few Shot.ipynb'   N_Shot_Lerning.ipynb   train.csv
 models		   test.csv		  validation.csv


In [ ]:
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install sacrebleu


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.7 MB/s eta 0:00:0

In [ ]:
import torch 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, Seq2SeqTrainer, TrainingArguments, DataCollatorForSeq2Seq, AutoModelForSequenceClassification, Seq2SeqTrainingArguments
import numpy as np
import evaluate
from datasets import load_dataset, Dataset
import pandas as pd


In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!nvidia-smi

Tue May  9 19:33:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model_name_or_path = "t5-small"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

In [ ]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
train_df = pd.read_csv("./train.csv")

get_random_sample = lambda x: x.iloc[np.random.randint(0, len(x))]
shots_list = [] # stores df for 1,2,4,8,16 shot training data
prev = None
for i in range(16):
  dummy = train_df.groupby('label').apply(get_random_sample)
  curr = dummy.reset_index(drop = True)
  if i != 0:
    curr = prev.append(curr, ignore_index = True)
  prev = curr
  if i == 0 or i == 1 or i == 3 or i == 7 or i == 15:
    shots_list.append(curr)


one_shot_train_df = shots_list[0]
two_shot_train_df = shots_list[1]
four_shot_train_df = shots_list[2]
eight_shot_train_df = shots_list[3]
sixteen_shot_train_df = shots_list[4]

<ipython-input-23-3d8a5b18bd57>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr = prev.append(curr, ignore_index = True)
<ipython-input-23-3d8a5b18bd57>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr = prev.append(curr, ignore_index = True)
<ipython-input-23-3d8a5b18bd57>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr = prev.append(curr, ignore_index = True)
<ipython-input-23-3d8a5b18bd57>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr = prev.append(curr, ignore_index = True)
<ipython-input-23-3d8a5b18bd57>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

In [ ]:
data_files = {"test": "test.csv", "validation": "validation.csv"}

raw_datasets = load_dataset("./", data_files = data_files)
raw_datasets["train"] = Dataset.from_pandas(one_shot_train_df)
print(raw_datasets)

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 700
    })
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 7
    })
})


In [ ]:
print(raw_datasets["train"][0])

{'label': 'AddToPlaylist', 'text': 'Add Visible Wings to my playlist entre los andes'}


In [ ]:
# def tokenize_function(example):
#     return tokenizer(example["label"] + "", example["text"], truncation=True, return_tensors = "pt", padding = True)
prefix = "intent classification: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["label"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(raw_datasets["train"].column_names)
datacollator = DataCollatorForSeq2Seq(tokenizer = tokenizer, return_tensors= "pt", padding=True)
print(tokenized_datasets["train"][0])

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

{'input_ids': [9508, 13774, 10, 2334, 13291, 2317, 15753, 7, 12, 82, 23424, 1721, 10381, 11, 15, 7, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [2334, 3696, 15800, 3350, 1]}


In [ ]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    # predictions, labels = eval_pred
    # decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # result = bleu.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    # result["gen_len"] = np.mean(prediction_lens)

    # return {k: round(v, 4) for k, v in result.items()}

        preds, labels = eval_pred
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
            # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [label.strip() for label in decoded_labels]
        decoded_labels = [[label] for label in decoded_labels]
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)

        return result


In [ ]:
training_args = Seq2SeqTrainingArguments(evaluation_strategy = "epoch", output_dir="./models",
                                  learning_rate=5e-4,
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  weight_decay=0.01,
                                  save_total_limit=3,
                                  num_train_epochs=3,
                                  predict_with_generate=True,
                                  fp16=True,)




In [ ]:
trainer = Seq2SeqTrainer(model = model, 
                  args = training_args, 
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["validation"],
                  data_collator = datacollator,
                  tokenizer = tokenizer,
                  compute_metrics = compute_metrics
                  )

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len,Gen Len
1,No log,7.295347,0.000000,"[0, 0, 0, 0]","[6195, 5600, 5046, 4506]","[0.0, 0.0, 0.0, 0.0]",1.000000,6195,700,13.258571
2,No log,5.405038,0.000000,"[0, 0, 0, 0]","[5230, 4624, 4040, 3465]","[0.0, 0.0, 0.0, 0.0]",1.000000,5230,700,11.504286
3,No log,4.215196,0.000000,"[0, 0, 0, 0]","[5665, 5015, 4383, 3759]","[0.0, 0.0, 0.0, 0.0]",1.000000,5665,700,12.252857


Trainer is attempting to log a value of "[0, 0, 0, 0]" of type <class 'list'> for key "eval/counts" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[6195, 5600, 5046, 4506]" of type <class 'list'> for key "eval/totals" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.0, 0.0, 0.0, 0.0]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0, 0, 0, 0]" of type <class 'list'> for key "eval/counts" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[5230, 4624, 4040, 3465]" of type <class 'list'> for key "eval/totals" as a scalar. This invocation

TrainOutput(global_step=21, training_loss=6.818818591889881, metrics={'train_runtime': 306.0756, 'train_samples_per_second': 0.069, 'train_steps_per_second': 0.069, 'total_flos': 80094855168.0, 'train_loss': 6.818818591889881, 'epoch': 3.0})